In [1]:
import os
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
from groq import Groq
import openai

In [2]:
import minsearch

## Ingestion

In [3]:
df = pd.read_csv('../data/data.csv')

In [15]:
documents = df.to_dict(orient='records')

In [16]:
index = minsearch.Index(
    text_fields=['exercise_name', 'type_of_activity', 'type_of_equipment', 'body_part',
       'type', 'muscle_groups_activated', 'instructions'],
    keyword_fields=['id']
)

In [17]:
index.fit(documents)

## Rag Flow

In [25]:
load_dotenv()

True

In [26]:
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

In [28]:
def search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [29]:
prompt_template = """
You're a fitness instructor. Answer the QUESTION based on the CONTEXT from our exercises database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

entry_template = """ 
exercise_name : {exercise_name}
type_of_activity : {type_of_activity}
type_of_equipment : {type_of_equipment}
body_part : {body_part}
type : {type}
muscle_groups_activated : {muscle_groups_activated}
instructions : {instructions}
""".strip()
    
def build_prompt(query, search_results):
    
    context = ""
    
    for doc in search_results:
        context = context + entry_template.format(**doc) + "\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt


In [30]:
search_results = search(query)
prompt = build_prompt(query, search_results)

In [31]:
def llm(prompt):
    response = client.chat.completions.create(
        model="meta-llama/llama-4-maverick-17b-128e-instruct",
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [32]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [35]:
query = "give me leg exercise for back posture"

In [36]:
answer =  rag(query)
print(answer)

To improve back posture, consider incorporating the "Deadlift (Kettlebell)" exercise into your routine. This exercise strengthens the hamstrings, glutes, and back, all of which play a crucial role in maintaining proper posture.

Alternatively, you can also try "Band Pull-Through" which targets the hamstrings, glutes, and back, helping to improve overall lower body strength and contribute to a better back posture.

Lastly, the "Deadlift" with a barbell also engages multiple muscle groups including the hamstrings, glutes, back, and core, making it an effective exercise for strengthening the muscles that support good back posture.
